in progress. todo:
- show actual network and compute loss value

In [ ]:
comment = '''

reference paper:
- Numerical Coordinate Regression with Convolutional Neural Networks - Aiden Nibali, ARVIX 2018
https://arxiv.org/abs/1801.07372


usage:
- one ECG image has 13 lead signal series 
- input image 1x3xHxW
- truth series 1xCxL (C=13)
- truth_mask 1xCxHxW (C=13)

coord = torch.arange(H)
y0 = torch.tensor([13 y0 values)] #zero mv line
pixel_to_mV = 1/(2*39.348837209302324)  


probability = UNET(rectified ECG image)  #softmax per pixel for 13+1 class, 1 is for background
#probability is 1x(C+1)xHxW
 
msel_loss, snr_loss = regression_loss(
	probability, coord, truth_series, 
	y0,  
	pixel_to_mV
)
js_loss = regularize_loss(probability, truth_mask)


'''

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# direct regression loss from mask probability
def normalize_prob(x, dim=2, eps=1e-8):
	# Ensure proper distributions that sum to 1 along `dim`
	s = x.sum(dim=dim, keepdim=True)
	return x / (s + eps)

def js_divergence(p, q, dim=-1, eps=1e-8):
	"""
	p, q: probability tensors (already normalized along `dim`)
	Returns JS divergence reduced over `dim` (keeps other dims).
	"""
	m = 0.5 * (p + q)
	kl_pm = (p * (torch.log(p + eps) - torch.log(m + eps))).sum(dim=dim)
	kl_qm = (q * (torch.log(q + eps) - torch.log(m + eps))).sum(dim=dim)
	return 0.5 * (kl_pm + kl_qm)

# Jensen-Shannon divergence
def regularize_loss(probability, truth): #todo: add mask

	EPS = 1e-6

	probability = probability[:, 1:] #exclude background
	B,C,H,W = probability.shape

	p = normalize_prob(probability, dim=2, eps=EPS)
	q = normalize_prob(truth, dim=2, eps=EPS)
	js = js_divergence(p, q, dim=2, eps=EPS)  # summed over `dim`

	js_loss = js.mean() #or js.sum()
	return js_loss

# ---------------------------------------------------------
# todo: mV to ??? to make regression target well-behave at backprop. std(target)=1
def regression_loss(
	probability, coord, truth,
	y0, #dc pulse
	pixel_to_mV
):
	# ECG signal = (y0-y)*pixel_to_mV
	# y0 is zero mV line

	EPS = 1e-6

	probability = probability[:, 1:] #exclude background
	B,C,L = truth.shape
	B,C,H,W = probability.shape
	coord = coord.reshape(1,1,H,1)

	probability = probability / (probability.sum(dim=2, keepdim=True) + EPS)
	predict = (y0-(probability*coord).sum(2))*pixel_to_mV  #B,C,W
	predict = F.interpolate(predict, size=L, mode='linear', align_corners=False) #B,C,L
	msel_loss = F.mse_loss(predict, truth)

	#---
	#snr
	signal = (truth)**2
	noise  = (predict-truth)**2
	snr = signal / (noise+EPS)
	snr_db =  10 * torch.log10(snr+EPS)
	snr_loss = -snr_db.mean()

	return msel_loss, snr_loss